In [24]:
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

In [2]:
#get datapath list
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [3]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase="train"
)
val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase="val"
)

In [7]:
#load dataset by batch
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

In [8]:
dataloaders_dict = {
    "train": train_dataloader,
    "val": val_dataloader
}

In [18]:
use_pretrained = torchvision.models.VGG16_Weights.DEFAULT
net = models.vgg16(weights=use_pretrained)

net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
net.train()

for name, _ in net.named_parameters():
    print(name)

features.0.weight
features.0.bias
features.2.weight
features.2.bias
features.5.weight
features.5.bias
features.7.weight
features.7.bias
features.10.weight
features.10.bias
features.12.weight
features.12.bias
features.14.weight
features.14.bias
features.17.weight
features.17.bias
features.19.weight
features.19.bias
features.21.weight
features.21.bias
features.24.weight
features.24.bias
features.26.weight
features.26.bias
features.28.weight
features.28.bias
classifier.0.weight
classifier.0.bias
classifier.3.weight
classifier.3.bias
classifier.6.weight
classifier.6.bias


In [14]:
criterion = nn.CrossEntropyLoss()

In [19]:
#fine tuning the model
params_to_update1, params_to_update2, params_to_update3 = [], [], []
update_list1 = [
    "classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"
]
update_list2 = [
    "classifier.6.weight", "classifier.6.bias"
]

for name, param in net.named_parameters():
    #update feature layer
    if "features" in name:
        param.requires_grad = True
        params_to_update1.append(param)
        print(f"{name} updated in param 1")
    elif name in update_list1:
        param.requires_grad = True
        params_to_update2.append(param)
        print(f"{name} updated in param 2")
    elif name in update_list2:
        param.requires_grad = True
        params_to_update3.append(param)
        print(f"{name} updated in param 3")
    else:
        param.requires_grad = False
        print(f"{name} not update when train")

features.0.weight updated in param 1
features.0.bias updated in param 1
features.2.weight updated in param 1
features.2.bias updated in param 1
features.5.weight updated in param 1
features.5.bias updated in param 1
features.7.weight updated in param 1
features.7.bias updated in param 1
features.10.weight updated in param 1
features.10.bias updated in param 1
features.12.weight updated in param 1
features.12.bias updated in param 1
features.14.weight updated in param 1
features.14.bias updated in param 1
features.17.weight updated in param 1
features.17.bias updated in param 1
features.19.weight updated in param 1
features.19.bias updated in param 1
features.21.weight updated in param 1
features.21.bias updated in param 1
features.24.weight updated in param 1
features.24.bias updated in param 1
features.26.weight updated in param 1
features.26.bias updated in param 1
features.28.weight updated in param 1
features.28.bias updated in param 1
classifier.0.weight updated in param 2
classif

In [21]:
optimizer = optim.SGD([
    {"params": params_to_update1, "lr": 1e-4},
    {"params": params_to_update2, "lr": 5e-4},
    {"params": params_to_update3, "lr": 1e-3},
], momentum=0.9
)

In [27]:
def train(net, dataloaders_dict, criterion, optimizer, num_epochs):
    device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
    device = torch.device("cpu")
    print(f"Device:{device}")

    net.to(device)

    #automatically choose best convolution algorithm
    #if input size == static, this will work
    #else may slow down
    torch.backends.cudnn.benchmark = True

    for epoch in range(num_epochs):
        print(f"Epoch: {epoch+1}/{num_epochs}", end="\n====================\n")
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            for inputs, labels in tqdm(dataloaders_dict[phase], colour="green"):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=="train"):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    preds = outputs.argmax(1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

            print(f"{phase} Loss: {epoch_loss/len(dataloaders_dict[phase].dataset):.4f} Acc: {epoch_corrects.double()/len(dataloaders_dict[phase].dataset):.4f}")

In [28]:
num_epochs = 5
train(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Device:cpu
Epoch: 1/5


  0%|          | 0/8 [00:00<?, ?it/s]